In [1]:
using StatsBase
using LinearAlgebra
using JuMP
using Gurobi
using CSV
using Distances
using SparseArrays
using Printf
using DataFrames

In [22]:
struct Object
    points::Matrix{Float64} # 2 x n matrix where each column is a point
    weights::Vector{Float64}
end
Base.length(p::Object) = length(p.weights)

function Base.rand(::Type{Object}; n_points = rand(1:5),
                                 points = rand(2,n_points),
                                use_rand = true)
    if use_rand
        weights = rand(n_points)
    else
        weights = ones(n_points)
    end
    return Object(points, weights)
end

# Destination and Source
D = rand(Object, n_points = 4, use_rand = false)
S = rand(Object, n_points = 4, use_rand = false)
cost = pairwise(Euclidean(), D.points, S.points; dims=2)

4×4 Array{Float64,2}:
 0.849429  0.609275  0.633754  0.66531 
 0.443114  0.478644  0.234312  0.301401
 0.805085  0.611142  0.404182  0.586697
 0.298409  0.153418  0.67766   0.322787

In [23]:
solCount = 1
m = JuMP.direct_model(Gurobi.Optimizer(PoolSearchMode=2, PoolSolutions=solCount, SolutionNumber=0))

@variable(m, X[axes(cost,1), axes(cost,2)] ≥ 0, Int)
@objective(m, Min, cost ⋅ X)
@constraint(m,sum(X) .== min(sum(D.weights), sum(S.weights)))
@constraint(m, X * ones(Int, length(D)) .<= S.weights)
@constraint(m, X'ones(Int, length(S)) .<= D.weights);
optimize!(m)
obj = objective_value(m)
solution = value.(X)

Optimize a model with 9 rows, 16 columns and 48 nonzeros
Variable types: 0 continuous, 16 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 8e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 4e+00]
Presolve time: 0.00s
Presolved: 9 rows, 16 columns, 48 nonzeros
Variable types: 0 continuous, 16 integer (16 binary)

Root relaxation: objective 1.613267e+00, 7 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       1.6132668    1.61327  0.00%     -    0s

Explored 0 nodes (7 simplex iterations) in 0.00 seconds
Thread count was 4 (of 4 available processors)

Solution count 1: 1.61327 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.613266755087e+00, best bound 1.613266755087e+00, gap 0.0000%


4×4 Array{Float64,2}:
 -0.0   1.0  -0.0   0.0
 -0.0  -0.0   0.0   1.0
 -0.0  -0.0   1.0   0.0
  1.0   0.0  -0.0  -0.0

Presented matrix is solution. By the intersection of column and raw we can see that which source will be connected with the destination

1TH SOURCE 2ND DESTINATION

2ND SOURCE 4TH DESTINATION

3RD SOURCE 3TH DESINATION

4TH SOURCE 1ST DESINATION